In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import statsmodels.formula.api as smf
warnings.filterwarnings('ignore')
data = pd.read_csv('train.csv')
train = data.sample(frac=0.7, random_state=1)
test = data[data.index.isin(train.index)==False]
#train['Obs']=range(1,805)  # maybe change index from 1 to 804
train.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SalePrice
8,20,RL,NaN,6853,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2009,WD,220000
791,160,FV,30.0,3180,Pave,Pave,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2006,WD,151000
571,20,RL,60.0,6960,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,Shed,500,11,2009,WD,120500
224,20,RL,70.0,8050,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,3,2007,WD,127000
498,20,RL,NaN,9556,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,8,2007,WD,160000


In [2]:
rows, columns = train.shape
print(rows)
print(columns)

563
79


In [3]:
train.describe().round(2) # without object type

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemod/Add,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,563.00,451.00,563.00,563.00,563.00,563.00,563.00,560.00,563.00,563.00,...,563.00,563.00,563.00,563.00,563.00,563.00,563.00,563.00,563.00,563.00
mean,56.86,68.29,9800.64,5.97,5.63,1970.00,1982.61,103.04,439.10,54.17,...,90.55,41.44,25.37,1.73,14.31,1.02,21.76,6.06,2007.91,173867.17
std,41.65,22.60,5832.44,1.32,1.08,27.55,20.78,189.73,410.48,186.25,...,121.59,59.17,64.31,17.14,49.66,24.28,223.55,2.58,1.30,69561.87
min,20.00,21.00,1300.00,2.00,3.00,1880.00,1950.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,2006.00,35000.00
25%,20.00,56.00,7490.00,5.00,5.00,1953.00,1963.50,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.00,2007.00,129000.00
50%,50.00,69.00,9259.00,6.00,5.00,1970.00,1990.00,0.00,404.00,0.00,...,0.00,16.00,0.00,0.00,0.00,0.00,0.00,6.00,2008.00,156000.00
75%,70.00,80.00,11366.50,7.00,6.00,1997.00,2002.00,148.00,695.50,0.00,...,166.00,62.00,0.00,0.00,0.00,0.00,0.00,7.00,2009.00,206500.00
max,190.00,155.00,70761.00,10.00,9.00,2008.00,2009.00,1224.00,1965.00,1526.00,...,646.00,365.00,364.00,219.00,287.00,576.00,4500.00,12.00,2010.00,538000.00


In [4]:
# variable preparation

predictors=list(train.columns[2:-1])
continuous_predictors=train.select_dtypes(['int64']).columns
columns_categorical=train.select_dtypes(['object']).columns


## Step 1: Categorial variables clean and discussion

In [5]:
train['MSZoning'].unique()

array(['RL', 'FV', 'RM', 'RH', 'C (all)', 'I (all)'], dtype=object)

In [6]:
# these variables are part of 'columns_categorical', 
categorical=['MSZoning', 'Street','LotShape', 'LandContour','Utilities','LotConfig', 'LandSlope'
           , 'Neighborhood', 'Condition1', 'Condition2' ,'BldgType','HouseStyle', 'RoofStyle', 'RoofMatl', 
             'Exterior1st','Exterior2nd','ExterQual', 'ExterCond', 'Foundation','Heating'
            , 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual','Functional'
            , 'PavedDrive','SaleType']


train['MSZoning']=train['MSZoning'].str.replace('(',"")
train['MSZoning']=train['MSZoning'].str.replace(')',"")
for item1 in categorical:
    train[item1]=train[item1].str.replace(' ',"")
    length=len(train[item1].unique())
    count=0
    while (length-1)>=count:
        for item2 in train[item1].unique():
            count1=str(count)
            train[item1]=train[item1].str.replace(item2,count1)
            count=int(count1)+1

In [7]:
#This is to replace NaN with empty string for now
train = train.replace(np.nan, '', regex=True)

#Use this to replace whatever first argument is with second argument
#train['Alley'] = train['Alley'].replace(0,'' , regex=True)


### Only 3 values with pool variables, we can just drop this

In [8]:
del train['PoolQC']
del train['PoolArea']
del test['PoolQC']
del test['PoolArea']

In [9]:
df = pd.DataFrame({'Fence':list(train['Fence'])})
pd.get_dummies(df).head()
#Here, the base case Fence means the house does not have a 

,Fence_,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0


23 places have a feature out of 805 observations. Is it worthwhile to even include? Only 2% of observations have this

Therefore, I decided to remove it.

In [10]:
del train['MiscFeature']
del test['MiscFeature']

In [11]:
train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,Fence,MiscVal,MoSold,YrSold,SaleType,SalePrice
8,20,0,,6853,0,,0,0,0,0,...,25,0,0,0,,0,6,2009,0,220000
791,160,1,30,3180,0,Pave,1,0,0,0,...,166,0,0,0,,0,6,2006,0,151000
571,20,0,60,6960,0,,1,0,0,0,...,0,0,0,0,,500,11,2009,0,120500
224,20,0,70,8050,0,,1,0,0,0,...,64,0,0,0,,0,3,2007,0,127000
498,20,0,,9556,0,,0,0,0,1,...,21,0,0,0,,0,8,2007,0,160000


In [12]:
del train['Alley']
del test['Alley']

In [13]:
#Need to conver any string into integer for machine learning algorithm to work
combined_set = pd.concat([train, test], axis = 0)

In [14]:
for feature in combined_set.columns: # Loop through all columns in the dataframe
    if combined_set[feature].dtype == 'object': # Only apply for columns with categorical strings
        combined_set[feature] = pd.Categorical(combined_set[feature]).codes # Replace strings with an integer

In [15]:
combined_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 804 entries, 8 to 802
Data columns (total 75 columns):
MSSubClass       804 non-null int64
MSZoning         804 non-null int8
LotFrontage      804 non-null int8
LotArea          804 non-null int64
Street           804 non-null int8
LotShape         804 non-null int8
LandContour      804 non-null int8
Utilities        804 non-null int8
LotConfig        804 non-null int8
LandSlope        804 non-null int8
Neighborhood     804 non-null int8
Condition1       804 non-null int8
Condition2       804 non-null int8
BldgType         804 non-null int8
HouseStyle       804 non-null int8
OverallQual      804 non-null int64
OverallCond      804 non-null int64
YearBuilt        804 non-null int64
YearRemod/Add    804 non-null int64
RoofStyle        804 non-null int8
RoofMatl         804 non-null int8
Exterior1st      804 non-null int8
Exterior2nd      804 non-null int8
MasVnrType       804 non-null int8
MasVnrArea       804 non-null int16
ExterQual    

In [16]:
final_train = combined_set[:train.shape[0]] # Up to the last initial training set row
final_test = combined_set[train.shape[0]:] # Past the last initial training set row

In [17]:
#This is the price variable we need to remove
y_train = final_train.pop('SalePrice')
y_test = final_test.pop('SalePrice')

In [18]:
import numpy
import pandas
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Time for some LASSO

In [19]:
from sklearn.linear_model import LassoCV

lasso = LassoCV(cv=10)
lasso.fit(final_train, np.ravel(y_train)) 

LassoCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
    max_iter=1000, n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)

In [20]:
#Getting the coefficients
pd.DataFrame(lasso.coef_.round(3), index = final_train.columns).T

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,Fence,MiscVal,MoSold,YrSold,SaleType
0,-0.0,-0.0,-0.0,0.995,-0.0,-0.0,0.0,-0.0,0.0,0.0,...,34.918,0.0,-0.0,-0.0,0.0,-0.0,-0.283,-0.0,0.0,0.0


# Time for Ridge Regression

In [21]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV

alphas = np.exp(np.linspace(-10,20,500)) 
ridge = RidgeCV(alphas=alphas, cv=5)
ridge.fit(final_train, np.ravel(y_train))
ridge = Ridge(alpha=ridge.alpha_)
ridge.fit(final_train, np.ravel(y_train))

Ridge(alpha=15.572257753236348, copy_X=True, fit_intercept=True,
   max_iter=None, normalize=False, random_state=None, solver='auto',
   tol=0.001)

In [22]:
pd.DataFrame(ridge.coef_.round(3), index = final_train.columns).T

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,Fence,MiscVal,MoSold,YrSold,SaleType
0,-49.938,-3313.516,-61.226,1.15,-378.438,-3055.39,3.716,-221.021,302.238,3407.903,...,7.868,18.371,-8.944,-53.209,46.214,-598.252,1.123,-124.793,201.883,1467.445


# Time for Elastic Net

In [23]:
from sklearn.linear_model import ElasticNetCV
enet = ElasticNetCV(l1_ratio=[0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.95,0.99], cv=5)
enet.fit(final_train, np.ravel(y_train))

from sklearn.linear_model import ElasticNet
enet = ElasticNet(alpha=enet.alpha_, l1_ratio=enet.l1_ratio_)
enet.fit(final_train, np.ravel(y_train))

ElasticNet(alpha=166961.3766010862, copy_X=True, fit_intercept=True,
      l1_ratio=0.98999999999999999, max_iter=1000, normalize=False,
      positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [24]:
pd.DataFrame(enet.coef_.round(3), index = final_train.columns).T

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,Fence,MiscVal,MoSold,YrSold,SaleType
0,0.0,-0.0,-0.0,0.92,-0.0,-0.0,0.0,-0.0,0.0,0.0,...,39.2,0.222,-17.891,-0.0,0.0,-0.0,-2.123,-0.0,0.0,0.0


In [25]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# Cross validator
kf=KFold(10, shuffle=True, random_state=1) 

# Methods
columns = ['CV RMSE']
rows = ['Ridge','Lasso', 'Elastic Net']

regressions = [ridge, lasso, enet]
results = pd.DataFrame(0.0, columns=columns, index=rows) # initialising a dataframe to hold the results

methods = {k: v for k, v in zip(rows, regressions)}

# Computing the results (Scikit Learn objects)
keys = ['Ridge','Lasso','Elastic Net']
for key in keys:
    scores = cross_val_score(methods[key], final_train, np.ravel(y_train), cv=kf, scoring = 'neg_mean_squared_error')
    results.loc[key] = np.sqrt(-1*np.mean(scores))

results.round(3)

,CV RMSE
Ridge,22350.562
Lasso,30721.526
Elastic Net,32268.624


In [27]:
columns=['Test RMSE', 'SE', 'Test R2', 'SE']
rows=['Ridge', 'Lasso', 'Elastic Net']
results=pd.DataFrame(0.0, columns=columns, index=rows) 

# from now on, we use the Jackknife method to be able to report the standard errors for the test results
from QBUS2820 import rmse_jack, r2_jack 

methods=[ridge, lasso, enet]
i=0
for method in methods:
    pred=method.predict(final_test)
    results.iloc[i,0], results.iloc[i,1] = rmse_jack(y_test, pred)
    results.iloc[i,2], results.iloc[i,3] = r2_jack(y_test, pred)
    i+=1

results.round(3)

,Test RMSE,SE,Test R2,SE
Ridge,50372.276,2508.451,0.497,0.065
Lasso,32733.989,4229.731,0.788,0.035
Elastic Net,34323.273,3971.281,0.766,0.035


In [28]:
columns=['Test RMSE', 'SE', 'Test R2', 'SE']
rows=['Ridge', 'Lasso', 'Elastic Net']
results=pd.DataFrame(0.0, columns=columns, index=rows) 

print(cross_val_score(lasso, final_train, y_train, cv=10))
print(cross_val_score(ridge, final_train, y_train, cv=10))
print(cross_val_score(enet, final_train, y_train, cv=10))

[ 0.85619295  0.68467377  0.65259311  0.8293292   0.89441665  0.82515952
  0.77731604  0.72997456  0.7774513   0.84076985]
[ 0.92374055  0.84700688  0.8937149   0.91129798  0.89967804  0.91608342
  0.89397134  0.86150762  0.84092677  0.9040065 ]
[ 0.82398782  0.66433726  0.60347725  0.81560381  0.88273668  0.81295084
  0.75468555  0.70062787  0.76053512  0.82098528]


# Appears that Lasso was the best prediction

## Combine all 3 to see what happens and other pairs

In [38]:
pred_R = ridge.predict(final_test)
pred_L = lasso.predict(final_test)
pred_E = enet.predict(final_test)

In [36]:
pred_final = (pred_R+pred_L+pred_E)/3
pred_final2 = (pred_L+pred_E)/2
pred_final3 = (pred_R+pred_E)/2
pred_final4 = (pred_L+pred_R)/2

columns=['Test RMSE', 'SE', 'Test R2', 'SE']
rows = ['Results with all 3', 'Lasso + Enet', 'Ridge + Enet', 'Lasso + Ridge']
results=pd.DataFrame(0.0, columns=columns, index=rows) 

results.iloc[0,0], results.iloc[0,1] = rmse_jack(y_test, pred_final)
results.iloc[0,2], results.iloc[0,3] = (r2_jack(y_test, pred_final))
results.iloc[1,0], results.iloc[1,1] = rmse_jack(y_test, pred_final2)
results.iloc[1,2], results.iloc[1,3] = (r2_jack(y_test, pred_final2))
results.iloc[2,0], results.iloc[2,1] = rmse_jack(y_test, pred_final3)
results.iloc[2,2], results.iloc[2,3] = (r2_jack(y_test, pred_final3))
results.iloc[3,0], results.iloc[3,1] = rmse_jack(y_test, pred_final4)
results.iloc[3,2], results.iloc[3,3] = (r2_jack(y_test, pred_final4))
results.round(3)

,Test RMSE,SE,Test R2,SE
Results with all 3,32378.989,4018.047,0.792,0.029
Lasso + Enet,33435.193,4106.365,0.778,0.035
Ridge + Enet,35152.234,3616.002,0.755,0.027
Lasso + Ridge,34670.560,3684.991,0.762,0.027


# Slight improvement when we combine all 3